In [ ]:
%matplotlib notebook
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

# Inferencia estadística con tests no-paramétricos

Hagamos un breve recordatorio de las lecciones anteriores para motivar este tema

## Ejemplo (recordatorio) de inferencia con test paramétrico

Digamos por ejemplo que queremos estimar los parámetros $\mu$ y $\sigma^2$ de la población

Colectamos nuestras muestras (observaciones) y construimos una estadístico muestral: 

$$
\bar X = \sum_i X_i/N
$$

Si **asumimos** que la población se distribuye normal entonces: $\bar X = \mathcal{N}(\mu, \sigma^2/N)$, es decir que 
- En promedio la media muestral equivale a la media de la población
- A mayor número de muestras menor es la varianza de nuestro estimador

Luego, para tomar decisiones basados en $\bar X$, usamos un estadístico de test: 

$$
t = \frac{\bar X - \mu}{\sqrt{\hat \sigma^2/N}}
$$ 

(o estadístico Z si $\sigma$ es conocido)

Dado que conocemos la distribución de $\bar X$ podemos calcular la distribución de $t$ bajo la hipótesis nula (e.g. $\mu=0$) y calcular el p-value 

> p-value: pbb de observar un valor más grande que t si la hipótesis nula fuera cierta

### El análisis descansa en los supuestos sobre la distribución de la población

¿Qué ocurre en el ejemplo anterior si el supuesto sobre la distribución de la población estaba equivocado?

La distribución muestral será erronea, la distribución nula será erronea y el p-value será erroneo

### Horror ¿Qué hago?

1. (Mantén la calma)
1. En la práctica los tests parámetricos funcionan ante "ligeras" desviaciones de los supuestos
1. Podemos **comprobar** los supuestos antes de hacer el análisis
1. Si los supuestos no se cumplen podemos usar **inferencia no paramétrica**

## Evaluando normalidad con test no-paraḿetrico

La normalidad es un supuesto en muchos procedimientos parámetricos, por ejemplo: t-test, ANOVA, regresion lineal

Tengamos en cuanta las características principales de la distribución normal

- Unimodal con media igual a su moda
- Simétrica en torno a la media
- Concentrada en torno a la media: ~68% a $\pm \sigma$, ~95% a $\pm 2\sigma$, ...

Si tenemos suficientes muestras podriamos observar el histograma y comprobar visualmente si esto se cumple

Observe las siguientes distribuciones, ¿Cuáles son no-normales? ¿Cuál o cuáles características de la distribución normal no se están cumpliendo?

In [ ]:
data1 = scipy.stats.norm(loc=4, scale=2).rvs(1000)
data2 = scipy.stats.gamma(a=2, scale=1).rvs(1000)
data3 = scipy.stats.uniform(loc=2, scale=1).rvs(1000)

fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    #data = (data - np.mean(data))/np.std(data)
    ax[i].hist(data, bins=10, density=True)
    ax[i].set_xlabel('data'+str(i))

### Momentos estadísticos de la distribución normal

En el caso particular de la normal sabemos que el tercer momento (simetría) debiese ser cero

Podemos calcular la simetría usando [`scipy.stats.skew`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html), para el caso particular de la normal puede ser util [`scipy.stats.skewtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skewtest.html)

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.skew(data))

El cuarto momento (kurtosis) también tiene un valor particular en el caso de la distribución normal

La función [`scipy.stats.kurtosis`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html) tiene dos definiciones de Kurtosis

La definición por defecto (Fisher) nos da cero en el caso de que la distribución sea normal

Puede ser también util [`scipy.stats.kurtosistest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosistest.html)

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.kurtosis(data))

### Probability plots (PP)

Otra opción gráfica son los *probability plots* 

Se calculan los cuantiles (qq) o percentiles (pp) de dos muestras **o** de una muestra y una distribución teórica

Luego se grafica uno en función del otro. Mientras más se parezca el resultado a una linea recta más similares son las distribuciones

Podemos usar [`scipy.stats.probplot`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot) para comparar una muestra con una distribución teórica 

- En este caso la distribución teórica será la normal estándar
- Dado que la prueba es basada en rangos no necesitamos entregar parámetros de localización (media) o escala (varianza)
- Cualquier parámetro de otro tipo (shape) se debe indicar




In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    #data = ((data - np.mean(data))/np.std(data)).copy()
    (osm, osr), (w, b, r2) = scipy.stats.probplot(data, dist="norm", fit=True);
    ax[i].plot(osm, osr)
    ax[i].plot(np.arange(-3, 4), np.arange(-3, 4)*w + b, 'k--', alpha=0.25)
    ax[i].set_title('data'+str(i));

### Test no-paramétrico de Kolmogorov-Smirnov (KS)

Este test examina la distancia entre las funciones de distribución acumulada (CDF) de dos muestras. Es un test no parámetrico, es decir que no asume una distribución particular para las muestras. Solo se puede usar para distribuciones univariadas continuas

- Si se compara una muestra empírica con una CDF teórica: Test de KS de una muestra
- Si se comparan dos muestras empíricas: Test de KS de dos muestras


#### Caso de una muestra

El estadístico de test para KS es

$$
D_n = \sup_x |F_n(x) - F(x)|
$$

donde $F(x) = \int_{-\infty}^x f(x) dx = P(X<x)$ es la CDF teórica y 

$$
F_n(x) = \frac{1}{N} \sum_{i=1}^N \mathbb{1}[X_i<x]
$$

es la distribución acumulada empírica (ECDF)

Veamos como se ve la ECDF de las tres muestras anteriores contra la CDF de una normal estándar

In [ ]:
def ecdf(data):
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x,y)

fig, ax = plt.subplots(1, 3, figsize=(7, 2.5), tight_layout=True)
for i, data in enumerate([data1, data2, data3]):
    # ECDF
    data = ((data - np.mean(data))/np.std(data)).copy() 
    x, y = ecdf(data)
    ax[i].plot(x, y)
    # CDF standard normal
    x_t = np.linspace(np.amin(data), np.amax(data), num=100)
    y_t = scipy.stats.norm.cdf(x_t)    
    ax[i].plot(x_t, y_t, 'k--', alpha=0.5)

> $D_n$ es la distancia más larga entre todas las distancias de la CDF teórica y empírica

Las hipótesis de la prueba KS de una muestra son

- $\mathcal{H}_0$: Los datos siguen la distribución teórica especificada
- $\mathcal{H}_A$: Los datos no siguen la distribución teórica especificada

Si $D_n$ es menor que el valor crítico no podemos rechazar $H_0$

Podemos usar [`scipy.stats.kstest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html#scipy.stats.kstest) para probar nuestros datos contra la distribución normal

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    data = ((data - np.mean(data))/np.std(data)).copy()
    Dn, pvalue = scipy.stats.kstest(data, scipy.stats.norm.cdf) #Acepta una cdf o un string, e.g. 'norm'
    print(i, Dn, pvalue)
    if pvalue < 0.05:
        display("Rechazo la hipótesis nula: nuestros datos no siguen la distribución especificada")

Problemas de Kolmogorov-Smirnov

- Solo sirve para distribuciones continuas y univariadas
- Es más sensible en el centro que en las colas de la distribución
- Se debe especificar completamente la distribución teórica

#### Opciones

Si mi distribución es discreta: Test de chi cuadrado

Mejoras de KS que le da más ponderación a las colas:
- [Test de Anderson-Darling](https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm), con implementación en [`scipy.stats.anderson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html)  (normal, exponencial y gumbel)
- [Test de Shapiro-Wilks](https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm) test con implementación en [`scipy.stats.shapiro`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html) (solo normal)

Si se quiere probar normalidad pero no se sabe la media o la varianza se puede usar el test de [Lilliefors](https://www.statsmodels.org/0.6.1/generated/statsmodels.stats.diagnostic.lillifors.html?highlight=lilliefors)


### Ejemplo, test de Anderson Darling con scipy:

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    #data = ((data - np.mean(data))/np.std(data)).copy() # no requiere estandarización
    statistic, critical_value, significance_level = scipy.stats.anderson(data, 'norm')
    print(i, statistic)
    if statistic > critical_value[significance_level==5.][0]:
        display("Rechazo la hipótesis nula: nuestros datos no siguen la distribución especificada")

## Inferencia no paramétrica

Inferencia: Estimar parámetros o probar hipótesis sobre una población

En inferencia **no parámetrica** usamos estadísticos cuya distribución **no depende de supuestos sobre la distribución de la población**

- No paramétrico $=$ No asumimos **distribución específica** para la población 
- No paramétrico $\neq$ Libre de supuestos (e.g. continuidad, muestras independientes)
- No paramétrico $\neq$ Sin parámetros (e.g. la cantidad de bines de un histograma)

Trade-off: 

- Si sus supuestos se cumplen, los métodos parámetricos son la ópcion de mayor poder estadístico (sensibilidad)
- Los métodos no-parámetricos tienen un poder igual o menor pero no requieren de supuestos

En clases anteriores vimos

- Histograma
- Kernel Density Estimation
- Bootstrap

En esta clase complementaremos con otras herramientas y tests no paramétricos

## Función cuantil y estadístico de orden


La CDF se define como

$$
P(X<x) = F(x) = p,
$$

donde $p\in [0, 1]$

En algunos casos nos interesa saber que valor $x$ corresponde a un valor dado $p$

Esto se hace con la función cuantil o CDF inversa

$$
Q(p) = \inf_{x\in \mathbb{R}} p \leq F(x)
$$

En scipy podemos calcular $x$ a partir de $p$ usando el atributo `ppf` (percent-point function) de una distribución 

In [ ]:
# Que valor de x corresponde a un 50% de la distribución?
display(scipy.stats.norm.ppf(0.5))
# y a un 99%?
display(scipy.stats.norm.ppf(0.99))

Sea ${X_1, X_2, X_3, \ldots, X_N}$ un conjunto de VAs iid y ${x_1, x_2, x_3, \ldots, x_N}$ una muestra aleatoria

Si asumimos que la población tiene una CDF continua (no hay dos VA con el mismo valor) entonces podemos hacer un ordenamiento único

$$
x_{(1)} < x_{(2)} < x_{(3)} < \ldots < x_{(N)} 
$$

que se llaman colectivamente el estadístico de orden de la muestra aleatoria y $x_{(r)}$ se llama el estadístico de orden r

Con este ordenamiento podemos construir la ECDF

$$
F_N(x) = \begin{cases} 0 & x < x_{(1)} \\ 
\frac{i}{N} & x_{(i-1)} \leq x < x_{(i)},  i=2,3,\ldots,N \\ 
1 & x_{(N)} < x\end{cases}
$$


In [ ]:
def ecdf(data):
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x,y)

### Propiedades de la ECDF

El ECDF es un estimador insesgado

$$
\mathbb{E}[F_N(x)] = F(x)
$$

y su varianza tiene a cero con $N$

$$
\text{Var}[F_N(x)] = \frac{1}{N} F(x) ( 1- F(x))
$$

Adicionalmente la ECDF

- es un estimador consistente de $F(x)$ (converge en probabilidad) 
- (estandarizada) es un asintoticamente normal estándar


### Función cuantil empírica

También podemos usar el estadístico de orden para construir la función cuantil empírica

$$
Q_N(u) = \begin{cases}
x_{(1)} &  0 < u \leq \frac{1}{N} \\
x_{(2)} & \frac{1}{N} < u  \leq\frac{2}{N} \\
\vdots & \vdots \\
x_{(N)} & \frac{(N-1)}{N} < u  \leq 1 \\
\end{cases}
$$

In [ ]:
data = np.random.randn(1000)
display(np.quantile(data, 0.5)) # NUMEROS ENTRE 0 Y 1
display(np.percentile(data, 50)) # NUMEROS ENTRE 0 Y 100

### Aplicaciones de estadísticos de orden

- El rango es $x_{N} - x_{1}$
- La mediana es $x_{(n/2)}$ (n par)

Los estadísticos de orden suelen ser más robustos que sus contrapartes en la presencia de *outliers*

Podemos calcular la mediana con `np.median`

In [ ]:
x = np.random.randn(10) # media cero
x[9] = 100
display(np.mean(x))
display(np.median(x))

Y el rango intercuartil, es decir la diferencia entre cuantil 0.75 y 0.25, usando `np.percentile`

In [ ]:
display(np.std(x))
display(np.subtract.reduce(np.quantile(x, [0.75, 0.25])))

## Tests no paramétricos

El test de KS que vimos antes es un ejemplo de test no paramétrico

Las propiedades de la ECDF nos permiten construir distribuciones nulas sin necesidad de asumir una distribución para la población

Existen muchos otros tests no-paramétricos, a continuación revisaremos algunos

### Test de Mann-Whitney U

Es una prueba no-paramétrica para comparar dos **muestras independientes**

> Muestras independientes: notas de niños y niñas en una prueba

> Muestras dependientes: notas de un mismo curso en dos pruebas

El objetivo es probar si provienen de una misma población (en función de su tendencia central)

Una forma parámetrica de probar esto es el *t-test* para comparar dos medias $\mu_0$ y $\mu_1$. Pero este test asume normalidad

El test de Mann-Whitney U en cambio no supone distribución para la población y algunos lo interpretan como una comparación entre medianas

La hipótesis nula es que no hay diferencia entre las distribuciones muestrales

> La prueba mezcla y ordena las observaciones de ambas muestras. Intutivamente, si las muestras son similares entonces sus observaciones deberían mezclarse en el ordenamiento. Si en cambio se clusterizan entonces las distribuciones son distintas

#### Algoritmo

1. Ordenar las observaciones de ambas muestras juntas
1. Sumar los rangos de la primera muestra $R_1$. $R_2$ queda definida ya que $R_1 + R_2 = \frac{N(N+1)}{2}$
1. Se calcula el estadístico $U_1 = R_1 - \frac{N_1(N_1+1)}{2}$, donde $N_1$ es el número de observaciones de la primera muestra. Por simetría $U_2$ queda inmediatamente especificado
1. Se usa $\min(U_1, U_2)$ para calcular el p-value

Esta prueba requiere al menos 20 observaciones en cada muestra para que sea efectiva

La prueba está implementada en [`scipy.stats.mannwhitneyu`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html)

### Test de los rangos con signo de Wilcoxon

Si las muestras que se quieren comparar son **dependientes** no se puede usar Mann Whitney U

Para esos casos está el test de rango con signo de Wilcoxon

Este test funciona como alternativa no-parámetrica al t-test (pareado) cuando los datos violan el supuesto de normalidad

Se usa para probar si hubo una diferencia significativa entre el "antes" y el "despues" de aplicar un tratamiento o intervención

Supuestos: 

- Los datos son continuos
- Los datos son pareados y vienen de la misma población
- Los pares son independientes y se escogen aleatoriamente



#### Algoritmo

Sea entonces $x_{11}, x_{12}, \ldots, x_{1N}$ y sus pares $x_{21}, x_{22}, \ldots, x_{2N}$

1. Se calculan las diferencias $z_i = x_{2i} - x_{1i}$
1. Se ordenan los valores absolutos de las diferencias $|z|_{(1)}, |z|_{(2)}, \ldots, |z|_{(N)}$ y se reserva el signo de cada diferencia 
1. Se suman los rangos de las diferencias con signo positivo: $W^{+}$
1. Se suman los rangos de las diferencias con signo negativo: $W^{-}$
1. El estadístico de prueba es $\min (W^{+}, W^{-})$ sobre el dataset reducido (se eliminan los $z_{(i)}=0$)

La hipótesis nula es que la mediana de las diferencias entre pares de observaciones es nula

> Intuitivamente, si la hipótesis nula es cierta, esperariamos que $W^{-}$ y $W^{+}$ sean similares. Si esto no se cumple se esperaría que $W^{+}$ fuera mayor que $W^{-}$

El test está implementado en [`scipy.stats.wilcoxon`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html)

### Otros tests no-parámetricos comunes

#### Test de Kruskall-Wallis: Alternativa a one-way ANOVA

Probar que existen diferencias significadas en una variable dependiente continua en función de una variable independiente categórica (2 o más grupos) 

Implementado en [`scipy.stats.kruskal`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html)

[Un digerido al respecto de esta y las pruebas que vimos anteriomente](https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/)



#### Kendal $\tau$: Alternativa al coeficiente de correlación de Pearson

Probar si dos variables son estadísticamente dependientes

Implementando en [`scipy.stats.kendalltau`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html)
 

## ¿Y que hago si mi estadístico es extraño y no hay un test que me sirva?

Puedes obtener la distribución muestral de tu estadístico usando bootstrap

Con esto se puede calcular intervalos de confianza y p-values empíricos

(Ver clase inferencia estadística)